In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read in K2 stellar parameters table
k2 = pd.read_csv('https://www.dropbox.com/scl/fi/ba9f1x2it1ukqar7xjlt8/k2all.csv?rlkey=a17ije9hsdle1t2ygr9v2izi5&st=rkx8dt2m&dl=1')

# Cast GaiaDR3 IDs as strings
k2['GaiaDR3'] = k2['GaiaDR3'].astype('|S').str.decode('utf-8')

k2

,EPIC_ID,Campaign,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,...,extinctionH,extinctionK,absJ,e_absJ,absH,e_absH,absK,e_absK,BC,e_BC
0,201121245,1,3598701493287932544,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,...,0.000,0.000,1.041,0.040,0.521,0.036,0.400,0.033,1.942,0.078
1,201122521,1,3598719154193454976,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,...,0.000,0.000,2.078,0.113,1.575,0.113,1.546,0.115,1.833,0.078
2,201123619,1,3598719497790839040,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,...,0.000,0.000,6.337,0.031,5.771,0.031,5.546,0.030,2.505,0.041
3,201124275,1,3598743206010226560,2MASS J11445381-0555420,176.224230,0.370010,-5.928365,0.230490,532.702515,5.769959,...,0.014,0.009,4.489,0.039,4.183,0.036,4.153,0.045,1.331,0.087
4,201126062,1,3598723620959410816,2MASS J11465755-0553007,176.739796,1.016033,-5.883488,0.543972,244.279556,3.241867,...,0.000,0.000,5.930,0.036,5.285,0.037,5.070,0.036,2.482,0.051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257482,212179420,18,678425788468626816,2MASS J08304609+2343144,127.692046,0.256701,23.720657,0.220915,929.733521,13.142090,...,0.014,0.009,2.456,0.040,2.183,0.039,2.130,0.041,1.207,0.066
257483,212179600,18,678250004049172480,2MASS J08274958+2343309,126.956631,0.220381,23.725236,0.169956,1289.393310,26.359250,...,0.000,0.000,0.617,0.054,0.111,0.054,0.025,0.053,1.939,0.091
257484,212179688,18,678241070517213056,2MASS J08263429+2343389,126.642887,0.320074,23.727465,0.213690,736.999084,8.799316,...,0.014,0.009,2.572,0.034,2.308,0.040,2.268,0.037,1.315,0.076
257485,212179696,18,678230212839882112,2MASS J08270421+2343393,126.767550,0.392856,23.727629,0.291679,225.309784,1.121964,...,0.000,0.000,-0.044,0.030,-0.523,0.049,-0.664,0.031,1.951,0.090


**Read in data from APOGEE, GALAH, and LAMOST**

Please be patient, it might take a while to download all the data depending on your internet connection.

In [3]:
# Read in APOGEE DR17 data
data = Table.read('https://portal.nersc.gov/project/cosmo/data/sdss/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStarLite-dr17-synspec_rev1.fits', format='fits', hdu=1)
del data['PARAMFLAG', 'X_H', 'X_H_ERR', 'X_M', 'X_M_ERR', 'ELEM_CHI2', 'ELEMFRAC', 'X_H_SPEC', 'X_M_SPEC']
df = data.to_pandas()

# Make some quality cuts
dfq = df[(df['SNR']>100)] #(df['ASPCAPFLAG']==0) &

# Drop targets with bad stellar parameters
dfq.dropna(subset=['TEFF', 'TEFF_ERR', 'LOGG', 'LOGG_ERR', 'M_H', 'M_H_ERR', 'FE_H', 'FE_H_ERR'],inplace=True)
dfq = dfq.sort_values(by='SNR',ascending=False)

# Keep highest SNR spectrum if multiples exist
dfq.drop_duplicates(subset='GAIAEDR3_SOURCE_ID',inplace=True)
dfq.reset_index(inplace=True,drop=True)

# Keep only the columns you need
apogee = dfq[['APOGEE_ID', 'TEFF', 'TEFF_ERR', 'LOGG',
       'LOGG_ERR', 'M_H', 'M_H_ERR', 'FE_H', 'FE_H_ERR', 
       'GAIAEDR3_SOURCE_ID']].copy()

# Rename columns
apogee = apogee.rename(columns={"GAIAEDR3_SOURCE_ID": "GaiaDR3","TEFF":"apogee_teff","TEFF_ERR":"apogee_e_teff",
                                "LOGG":"apogee_logg","LOGG_ERR":"apogee_e_logg","FE_H":"apogee_fe_h","FE_H_ERR":"apogee_e_fe_h",
                                "M_H":"apogee_m_h","M_H_ERR":"apogee_e_m_h"})
apogee.dropna(inplace=True)
apogee.reset_index(inplace=True,drop=True)

# Cast names as strings
apogee['APOGEE_ID'] = apogee['APOGEE_ID'].str.decode('utf-8')
apogee['GaiaDR3'] = apogee['GaiaDR3'].astype('|S').str.decode('utf-8')

# Merge with K2 data
k2 = k2.merge(apogee,on='GaiaDR3',how='left')
k2

,EPIC_ID,Campaign,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,...,e_BC,APOGEE_ID,apogee_teff,apogee_e_teff,apogee_logg,apogee_e_logg,apogee_m_h,apogee_e_m_h,apogee_fe_h,apogee_e_fe_h
0,201121245,1,3598701493287932544,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,...,0.078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201122521,1,3598719154193454976,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,...,0.078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201123619,1,3598719497790839040,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,...,0.041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201124275,1,3598743206010226560,2MASS J11445381-0555420,176.224230,0.370010,-5.928365,0.230490,532.702515,5.769959,...,0.087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201126062,1,3598723620959410816,2MASS J11465755-0553007,176.739796,1.016033,-5.883488,0.543972,244.279556,3.241867,...,0.051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257482,212179420,18,678425788468626816,2MASS J08304609+2343144,127.692046,0.256701,23.720657,0.220915,929.733521,13.142090,...,0.066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257483,212179600,18,678250004049172480,2MASS J08274958+2343309,126.956631,0.220381,23.725236,0.169956,1289.393310,26.359250,...,0.091,2M08274958+2343309,4858.410645,8.903093,3.030872,0.023333,-0.16629,0.00619,-0.16397,0.007207
257484,212179688,18,678241070517213056,2MASS J08263429+2343389,126.642887,0.320074,23.727465,0.213690,736.999084,8.799316,...,0.076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257485,212179696,18,678230212839882112,2MASS J08270421+2343393,126.767550,0.392856,23.727629,0.291679,225.309784,1.121964,...,0.090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Read in GALAH DR3 data
dr3_main = Table.read('https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/GALAH_DR3_main_allstar_v2.fits', format='fits', hdu=1)
dr3 = np.array(dr3_main['star_id', 'sobject_id', 'dr2_source_id', 'dr3_source_id', 'survey_name', 'field_id', 'flag_repeat', 'wg4_field', 'wg4_pipeline', 'flag_sp', 'teff', 'e_teff', 'irfm_teff', 'irfm_ebv', 'irfm_ebv_ref', 'logg', 'e_logg', 'fe_h', 'e_fe_h', 'flag_fe_h', 'fe_h_atmo', 'vmic', 'vbroad', 'e_vbroad', 'chi2_sp', 'alpha_fe', 'e_alpha_fe', 'nr_alpha_fe', 'flag_alpha_fe', 'flux_A_Fe', 'chi_A_Fe', 'Li_fe', 'e_Li_fe', 'nr_Li_fe', 'flag_Li_fe', 'C_fe', 'e_C_fe', 'nr_C_fe', 'flag_C_fe', 'O_fe', 'e_O_fe', 'nr_O_fe', 'flag_O_fe', 'Na_fe', 'e_Na_fe', 'nr_Na_fe', 'flag_Na_fe', 'Mg_fe', 'e_Mg_fe', 'nr_Mg_fe', 'flag_Mg_fe', 'Al_fe', 'e_Al_fe', 'nr_Al_fe', 'flag_Al_fe', 'Si_fe', 'e_Si_fe', 'nr_Si_fe', 'flag_Si_fe', 'K_fe', 'e_K_fe', 'nr_K_fe', 'flag_K_fe', 'Ca_fe', 'e_Ca_fe', 'nr_Ca_fe', 'flag_Ca_fe', 'Sc_fe', 'e_Sc_fe', 'nr_Sc_fe', 'flag_Sc_fe', 'Sc2_fe', 'e_Sc2_fe', 'nr_Sc2_fe', 'flag_Sc2_fe', 'Ti_fe', 'e_Ti_fe', 'nr_Ti_fe', 'flag_Ti_fe', 'Ti2_fe', 'e_Ti2_fe', 'nr_Ti2_fe', 'flag_Ti2_fe', 'V_fe', 'e_V_fe', 'nr_V_fe', 'flag_V_fe', 'Cr_fe', 'e_Cr_fe', 'nr_Cr_fe', 'flag_Cr_fe', 'Cr2_fe', 'e_Cr2_fe', 'nr_Cr2_fe', 'flag_Cr2_fe', 'Mn_fe', 'e_Mn_fe', 'nr_Mn_fe', 'flag_Mn_fe', 'Co_fe', 'e_Co_fe', 'nr_Co_fe', 'flag_Co_fe', 'Ni_fe', 'e_Ni_fe', 'nr_Ni_fe', 'flag_Ni_fe', 'Cu_fe', 'e_Cu_fe', 'nr_Cu_fe', 'flag_Cu_fe', 'Zn_fe', 'e_Zn_fe', 'nr_Zn_fe', 'flag_Zn_fe', 'Rb_fe', 'e_Rb_fe', 'nr_Rb_fe', 'flag_Rb_fe', 'Sr_fe', 'e_Sr_fe', 'nr_Sr_fe', 'flag_Sr_fe', 'Y_fe', 'e_Y_fe', 'nr_Y_fe', 'flag_Y_fe', 'Zr_fe', 'e_Zr_fe', 'nr_Zr_fe', 'flag_Zr_fe', 'Mo_fe', 'e_Mo_fe', 'nr_Mo_fe', 'flag_Mo_fe', 'Ru_fe', 'e_Ru_fe', 'nr_Ru_fe', 'flag_Ru_fe', 'Ba_fe', 'e_Ba_fe', 'nr_Ba_fe', 'flag_Ba_fe', 'La_fe', 'e_La_fe', 'nr_La_fe', 'flag_La_fe', 'Ce_fe', 'e_Ce_fe', 'nr_Ce_fe', 'flag_Ce_fe', 'Nd_fe', 'e_Nd_fe', 'nr_Nd_fe', 'flag_Nd_fe', 'Sm_fe', 'e_Sm_fe', 'nr_Sm_fe', 'flag_Sm_fe', 'Eu_fe', 'e_Eu_fe', 'nr_Eu_fe', 'flag_Eu_fe', 'ra_dr2', 'dec_dr2', 'parallax_dr2', 'parallax_error_dr2', 'r_est_dr2', 'r_lo_dr2', 'r_hi_dr2', 'r_len_dr2', 'rv_galah', 'e_rv_galah', 'rv_gaia_dr2', 'e_rv_gaia_dr2', 'red_flag', 'ebv', 'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf', 'flag_guess', 'v_jk', 'j_m', 'j_msigcom', 'h_m', 'h_msigcom', 'ks_m', 'ks_msigcom', 'ph_qual_tmass', 'w2mpro', 'w2mpro_error', 'ph_qual_wise', 'a_ks', 'e_a_ks', 'bc_ks', 'ruwe_dr2'])
df = pd.DataFrame(dr3,columns=['star_id', 'sobject_id', 'dr2_source_id', 'dr3_source_id', 'survey_name', 'field_id', 'flag_repeat', 'wg4_field', 'wg4_pipeline', 'flag_sp', 'teff', 'e_teff', 'irfm_teff', 'irfm_ebv', 'irfm_ebv_ref', 'logg', 'e_logg', 'fe_h', 'e_fe_h', 'flag_fe_h', 'fe_h_atmo', 'vmic', 'vbroad', 'e_vbroad', 'chi2_sp', 'alpha_fe', 'e_alpha_fe', 'nr_alpha_fe', 'flag_alpha_fe', 'flux_A_Fe', 'chi_A_Fe', 'Li_fe', 'e_Li_fe', 'nr_Li_fe', 'flag_Li_fe', 'C_fe', 'e_C_fe', 'nr_C_fe', 'flag_C_fe', 'O_fe', 'e_O_fe', 'nr_O_fe', 'flag_O_fe', 'Na_fe', 'e_Na_fe', 'nr_Na_fe', 'flag_Na_fe', 'Mg_fe', 'e_Mg_fe', 'nr_Mg_fe', 'flag_Mg_fe', 'Al_fe', 'e_Al_fe', 'nr_Al_fe', 'flag_Al_fe', 'Si_fe', 'e_Si_fe', 'nr_Si_fe', 'flag_Si_fe', 'K_fe', 'e_K_fe', 'nr_K_fe', 'flag_K_fe', 'Ca_fe', 'e_Ca_fe', 'nr_Ca_fe', 'flag_Ca_fe', 'Sc_fe', 'e_Sc_fe', 'nr_Sc_fe', 'flag_Sc_fe', 'Sc2_fe', 'e_Sc2_fe', 'nr_Sc2_fe', 'flag_Sc2_fe', 'Ti_fe', 'e_Ti_fe', 'nr_Ti_fe', 'flag_Ti_fe', 'Ti2_fe', 'e_Ti2_fe', 'nr_Ti2_fe', 'flag_Ti2_fe', 'V_fe', 'e_V_fe', 'nr_V_fe', 'flag_V_fe', 'Cr_fe', 'e_Cr_fe', 'nr_Cr_fe', 'flag_Cr_fe', 'Cr2_fe', 'e_Cr2_fe', 'nr_Cr2_fe', 'flag_Cr2_fe', 'Mn_fe', 'e_Mn_fe', 'nr_Mn_fe', 'flag_Mn_fe', 'Co_fe', 'e_Co_fe', 'nr_Co_fe', 'flag_Co_fe', 'Ni_fe', 'e_Ni_fe', 'nr_Ni_fe', 'flag_Ni_fe', 'Cu_fe', 'e_Cu_fe', 'nr_Cu_fe', 'flag_Cu_fe', 'Zn_fe', 'e_Zn_fe', 'nr_Zn_fe', 'flag_Zn_fe', 'Rb_fe', 'e_Rb_fe', 'nr_Rb_fe', 'flag_Rb_fe', 'Sr_fe', 'e_Sr_fe', 'nr_Sr_fe', 'flag_Sr_fe', 'Y_fe', 'e_Y_fe', 'nr_Y_fe', 'flag_Y_fe', 'Zr_fe', 'e_Zr_fe', 'nr_Zr_fe', 'flag_Zr_fe', 'Mo_fe', 'e_Mo_fe', 'nr_Mo_fe', 'flag_Mo_fe', 'Ru_fe', 'e_Ru_fe', 'nr_Ru_fe', 'flag_Ru_fe', 'Ba_fe', 'e_Ba_fe', 'nr_Ba_fe', 'flag_Ba_fe', 'La_fe', 'e_La_fe', 'nr_La_fe', 'flag_La_fe', 'Ce_fe', 'e_Ce_fe', 'nr_Ce_fe', 'flag_Ce_fe', 'Nd_fe', 'e_Nd_fe', 'nr_Nd_fe', 'flag_Nd_fe', 'Sm_fe', 'e_Sm_fe', 'nr_Sm_fe', 'flag_Sm_fe', 'Eu_fe', 'e_Eu_fe', 'nr_Eu_fe', 'flag_Eu_fe', 'ra_dr2', 'dec_dr2', 'parallax_dr2', 'parallax_error_dr2', 'r_est_dr2', 'r_lo_dr2', 'r_hi_dr2', 'r_len_dr2', 'rv_galah', 'e_rv_galah', 'rv_gaia_dr2', 'e_rv_gaia_dr2', 'red_flag', 'ebv', 'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf', 'flag_guess', 'v_jk', 'j_m', 'j_msigcom', 'h_m', 'h_msigcom', 'ks_m', 'ks_msigcom', 'ph_qual_tmass', 'w2mpro', 'w2mpro_error', 'ph_qual_wise', 'a_ks', 'e_a_ks', 'bc_ks', 'ruwe_dr2'])

# Set quality cuts
unflagged_stellar_parameters = (dr3_main['flag_sp'] == 0)
dfc = df[df['flag_sp']==0]
dfc = dfc[(dfc['snr_c1_iraf']>20) | (dfc['snr_c2_iraf']>20) | (dfc['snr_c3_iraf']>20) | (dfc['snr_c4_iraf']>20)]
dfc.reset_index(inplace=True,drop=True)

# Keep only the columns you need
galah = dfc[['star_id', 'dr2_source_id', 'dr3_source_id', 'teff', 'e_teff', 'logg', 'e_logg', 'fe_h', 'e_fe_h']].copy()
galah = galah.rename(columns={"dr2_source_id": "GaiaDR2","dr3_source_id":"GaiaDR3","star_id": "GALAH_ID", "teff": "galah_teff", "e_teff": "galah_e_teff", "logg": "galah_logg", "e_logg": "galah_e_logg",
                     "fe_h": "galah_fe_h", "e_fe_h": "galah_e_fe_h"})
galah.dropna(inplace=True)
galah.reset_index(inplace=True,drop=True)

# Fix the utf encoding b string for the star_id
galah['GALAH_ID'] = galah['GALAH_ID'].str.decode('utf-8')
galah['GaiaDR3'] = galah['GaiaDR3'].astype('|S').str.decode('utf-8')

# Merge with K2 data
k2 = k2.merge(galah,on=['GaiaDR3'],how='left')
k2

,EPIC_ID,Campaign,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,...,apogee_fe_h,apogee_e_fe_h,GALAH_ID,GaiaDR2,galah_teff,galah_e_teff,galah_logg,galah_e_logg,galah_fe_h,galah_e_fe_h
0,201121245,1,3598701493287932544,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201122521,1,3598719154193454976,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201123619,1,3598719497790839040,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201124275,1,3598743206010226560,2MASS J11445381-0555420,176.224230,0.370010,-5.928365,0.230490,532.702515,5.769959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201126062,1,3598723620959410816,2MASS J11465755-0553007,176.739796,1.016033,-5.883488,0.543972,244.279556,3.241867,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257482,212179420,18,678425788468626816,2MASS J08304609+2343144,127.692046,0.256701,23.720657,0.220915,929.733521,13.142090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257483,212179600,18,678250004049172480,2MASS J08274958+2343309,126.956631,0.220381,23.725236,0.169956,1289.393310,26.359250,...,-0.16397,0.007207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257484,212179688,18,678241070517213056,2MASS J08263429+2343389,126.642887,0.320074,23.727465,0.213690,736.999084,8.799316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257485,212179696,18,678230212839882112,2MASS J08270421+2343393,126.767550,0.392856,23.727629,0.291679,225.309784,1.121964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Read in LAMOST DR9 AFGK data
df = pd.read_csv('https://www.lamost.org/dr9/v2.0/catdl?name=dr9_v2.0_LRS_stellar.csv.gz', compression='gzip', header=0, sep=',', dtype={'gaia_source_id': str})

# Set quality cuts
dfq = df[(df['snrg']>20) & (df['snrg']<999)]
dfq = dfq[(dfq.teff_err != -9999) | (dfq.logg_err != -9999) | (dfq.feh_err != -9999)]
dfq = dfq.sort_values(by='snrg',ascending=False)
dfq.drop_duplicates(subset='gaia_source_id',inplace=True)
dfq.reset_index(inplace=True,drop=True)

# Keep only the columns you need
lamostafgk = dfq[['obsid','gaia_source_id','subclass','teff','teff_err','logg','logg_err','feh','feh_err']].copy()
lamostafgk.rename(columns={'obsid':'LAMOST_ID','subclass':'lamost_SpT','teff':'lamost_teff','teff_err':'lamost_e_teff','logg':'lamost_logg','logg_err':'lamost_e_logg','feh':'lamost_fe_h','feh_err':'lamost_e_fe_h','gaia_source_id':'GaiaDR3'},inplace=True)

# Drop non-Gaia targets
lamostafgk = lamostafgk[lamostafgk['GaiaDR3'].str.contains("nan")==False]
lamostafgk['GaiaDR3'] = lamostafgk['GaiaDR3'].astype('|S').str.decode('utf-8')

# Merge with K2 data
k2 = k2.merge(lamostafgk,on='GaiaDR3',how='left')
k2

,EPIC_ID,Campaign,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,...,galah_fe_h,galah_e_fe_h,LAMOST_ID,lamost_SpT,lamost_teff,lamost_e_teff,lamost_logg,lamost_e_logg,lamost_fe_h,lamost_e_fe_h
0,201121245,1,3598701493287932544,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,...,NaN,NaN,866601018.0,K0,4756.32,26.86,3.143,0.035,-0.241,0.019
1,201122521,1,3598719154193454976,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,...,NaN,NaN,528401010.0,G7,5151.36,40.72,3.804,0.055,-0.412,0.038
2,201123619,1,3598719497790839040,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201124275,1,3598743206010226560,2MASS J11445381-0555420,176.224230,0.370010,-5.928365,0.230490,532.702515,5.769959,...,NaN,NaN,502201005.0,F0,5817.48,35.03,3.965,0.050,-1.866,0.030
4,201126062,1,3598723620959410816,2MASS J11465755-0553007,176.739796,1.016033,-5.883488,0.543972,244.279556,3.241867,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257482,212179420,18,678425788468626816,2MASS J08304609+2343144,127.692046,0.256701,23.720657,0.220915,929.733521,13.142090,...,NaN,NaN,202803236.0,F5,6270.06,33.87,4.251,0.048,-0.495,0.029
257483,212179600,18,678250004049172480,2MASS J08274958+2343309,126.956631,0.220381,23.725236,0.169956,1289.393310,26.359250,...,NaN,NaN,144907096.0,K1,4891.87,52.62,3.166,0.080,-0.188,0.054
257484,212179688,18,678241070517213056,2MASS J08263429+2343389,126.642887,0.320074,23.727465,0.213690,736.999084,8.799316,...,NaN,NaN,202810102.0,F7,6124.84,36.38,4.061,0.052,-0.147,0.031
257485,212179696,18,678230212839882112,2MASS J08270421+2343393,126.767550,0.392856,23.727629,0.291679,225.309784,1.121964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Read in LAMOST DR9 M data
df = pd.read_csv('https://www.lamost.org/dr9/v2.0/catdl?name=dr9_v2.0_LRS_mstellar.csv.gz', compression='gzip', header=0, sep=',', dtype={'gaia_source_id': str})

# Set quality cuts
dfq = df[(df['snri']>20) & (df['snri']<999)]
dfq = dfq[(dfq.teff_err != -9999) | (dfq.logg_err != -9999) | (dfq.m_h_err != -9999)]
dfq = dfq.sort_values(by='snri',ascending=False)
dfq.drop_duplicates(subset='gaia_source_id',inplace=True)
dfq.reset_index(inplace=True,drop=True)

# Keep only the columns you need
lamostm = dfq[['obsid','gaia_source_id','subclass','teff','teff_err','logg','logg_err','m_h','m_h_err']].copy()
lamostm.rename(columns={'obsid':'LAMOSTM_ID','subclass':'lamostm_SpT','teff':'lamostm_teff','teff_err':'lamostm_e_teff','logg':'lamostm_logg','logg_err':'lamostm_e_logg','m_h':'lamostm_m_h','m_h_err':'lamostm_e_m_h','gaia_source_id':'GaiaDR3'},inplace=True)

# Drop non-Gaia targets
lamostm = lamostm[(lamostm['GaiaDR3'].str.contains("nan")==False) & (lamostm['GaiaDR3'].str.contains("-9999")==False)]
lamostm['GaiaDR3'] = lamostm['GaiaDR3'].astype('|S').str.decode('utf-8')

# Merge with K2 data
k2 = k2.merge(lamostm,on='GaiaDR3',how='left')
k2

,EPIC_ID,Campaign,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,...,lamost_fe_h,lamost_e_fe_h,LAMOSTM_ID,lamostm_SpT,lamostm_teff,lamostm_e_teff,lamostm_logg,lamostm_e_logg,lamostm_m_h,lamostm_e_m_h
0,201121245,1,3598701493287932544,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,...,-0.241,0.019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201122521,1,3598719154193454976,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,...,-0.412,0.038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201123619,1,3598719497790839040,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201124275,1,3598743206010226560,2MASS J11445381-0555420,176.224230,0.370010,-5.928365,0.230490,532.702515,5.769959,...,-1.866,0.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201126062,1,3598723620959410816,2MASS J11465755-0553007,176.739796,1.016033,-5.883488,0.543972,244.279556,3.241867,...,NaN,NaN,528401100.0,dM0,3870.8,100.29,5.069,0.21,-0.4307,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257482,212179420,18,678425788468626816,2MASS J08304609+2343144,127.692046,0.256701,23.720657,0.220915,929.733521,13.142090,...,-0.495,0.029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257483,212179600,18,678250004049172480,2MASS J08274958+2343309,126.956631,0.220381,23.725236,0.169956,1289.393310,26.359250,...,-0.188,0.054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257484,212179688,18,678241070517213056,2MASS J08263429+2343389,126.642887,0.320074,23.727465,0.213690,736.999084,8.799316,...,-0.147,0.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257485,212179696,18,678230212839882112,2MASS J08270421+2343393,126.767550,0.392856,23.727629,0.291679,225.309784,1.121964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
